In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
import random
import sys
import re

In [ ]:
# Load dataset (Example: Shakespeare Sonnets)
path = tf.keras.utils.get_file("shakespeare.txt", "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")
with open(path, "r", encoding="utf-8") as f:
    text = f.read().lower()


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Prepare character mapping
chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

# Create sequences
seq_length = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - seq_length, step):
    sentences.append(text[i: i + seq_length])
    next_chars.append(text[i + seq_length])

X = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1


In [ ]:
# Define RNN model
model = Sequential([
    Input(shape=(seq_length, len(chars))),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 2.5751
Epoch 2/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 1.9803
Epoch 3/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 1.7911
Epoch 4/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 1.6867
Epoch 5/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 1.6249
Epoch 6/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 1.5754
Epoch 7/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 1.5335
Epoch 8/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 1.5049
Epoch 9/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 1.4788
Epoch 10/10
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 1.4592


In [ ]:
# Function for text generation
def generate_text(seed_text, length=200, temperature=1.0):
    generated = seed_text
    for _ in range(length):
        x_pred = np.zeros((1, seq_length, len(chars)))
        for t, char in enumerate(seed_text):
            x_pred[0, t, char_to_idx[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        preds = np.asarray(preds).astype("float64")
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_index = np.random.choice(len(chars), p=preds)
        next_char = idx_to_char[next_index]
        generated += next_char
        seed_text = seed_text[1:] + next_char
    return generated

In [ ]:
# Example text generation
seed = "shall i compare thee to a summer's day? "
print(generate_text(seed))

shall i compare thee to a summer's day? go.
go, should stil his tome, what, i'll came
their wornistith: his iest a feire him: and
to him from so suck, a dufe like in my sunder,
our baurmand asteds to stene andervice,
but his out my such our
